# GPT Remove Particle Elements
---

In [1]:
from distgen import Generator

from gpt import GPT
from gpt.element import Screen
from gpt.lattice import Lattice
from gpt.remove_particles import Aperture
from gpt.remove_particles import CircularAperture

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

## Circular Aperture
This object uses the more general GPT Aperture object, and assumes cylindrical symmetry.

In [2]:
lat = Lattice('aperture_test')
lat.add(CircularAperture('a1', 1e-2, Rmax = 5e-2), ds=0.1)
lat.add(Screen('scr'), ds=0.25, ref_element='beg')

In [3]:
_=lat.write_gpt_lines(output_file='aperture.in')

In [4]:
distgen_yaml = """
n_particle: 1000
total_charge: 0 C
random:
    type: hammersley
r_dist:
    type: ru
    max_r: 2 cm
start:
    type: time
"""

In [5]:
D = Generator(distgen_yaml)
P = D.run()
P.pz = 1e3

AssertionError: Required generator parameter species not found.

In [ ]:
G = GPT('aperture.in', initial_particles=P)
G.set_variables({'ZSTOP':0.25, 'tmax':50e-8, 'Ntout': 200})
G.run()

In [ ]:
print(G)

In [ ]:
ax = plt.gca()
for ii in range(1, D['n_particle']+1):
    tii = G.trajectory(ii, data_type='tout')
    ax.plot(tii['z'], tii['x'])
ax = lat.plot_floor()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for ii in range(1, D['n_particle']):
    tii = G.trajectory(ii, data_type='tout')
    if(tii['z'][-1]>0.1):
        ax.plot(tii['x']*1000, tii['y']*1000, tii['z'], zdir='x', alpha=0.5)
    else:
        ax.plot(tii['x']*1000, tii['y']*1000, tii['z'], zdir='x', alpha=0.1)
    
ax.set_xlabel('z (m)');
ax.set_ylabel('x (mm)');
ax.set_zlabel('y (mm)');
ax.set_ylim([-20, 20])
ax.set_zlim([-20, 20])

In [ ]:
plt.plot(G.tout_stat('mean_z'), +G.tout_stat('sigma_x'))
plt.plot(G.tout_stat('mean_z'), +G.tout_stat('max_x'), '--k', alpha=0.2)
plt.plot(G.tout_stat('mean_z'), -G.tout_stat('sigma_x'), color='tab:blue');
plt.plot(G.tout_stat('mean_z'), +G.tout_stat('max_x'), '--k', alpha=0.2)
plt.plot(G.tout_stat('mean_z'), -G.tout_stat('max_x'), '--k', alpha=0.2);
plt.xlabel('z (m)')
plt.ylabel(r'$\sigma_{x,y}$, max[$x$, $y$] (m)')
plt.legend([r'$\sigma_{x,y}$', r'max[$x$, $y$]'])

In [ ]:
os.remove('aperture.in')

## Aperture
This wraps the more General GPT Aperture object and allows for elliptical aperture shape or providing a bitmap file.

In [ ]:
lat2 = Lattice('aperture_test')
lat2.add(Aperture('a1', a=0.01, b=0.01) , ds=0.1)
lat2.add(Screen('scr'), ds=0.25, ref_element='beg')

In [ ]:
_=lat2.write_gpt_lines(output_file='aperture2.in')

In [ ]:
G2 = GPT('aperture2.in', initial_particles=P)
G2.set_variables({'ZSTOP':0.25, 'tmax':50e-8, 'Ntout': 200})
G2.run()

In [ ]:
ax = plt.gca()
for ii in range(1, D['n_particle']+1):
    tii = G2.trajectory(ii, data_type='tout')
    ax.plot(tii['z'], tii['x'])
ax = lat.plot_floor()

In [ ]:
plt.plot(G2.tout_stat('mean_z'), +G2.tout_stat('sigma_x'))
plt.plot(G2.tout_stat('mean_z'), +G2.tout_stat('max_x'), '--k', alpha=0.2)
plt.plot(G2.tout_stat('mean_z'), -G2.tout_stat('sigma_x'), color='tab:blue');
plt.plot(G2.tout_stat('mean_z'), +G2.tout_stat('max_x'), '--k', alpha=0.2)
plt.plot(G2.tout_stat('mean_z'), -G2.tout_stat('max_x'), '--k', alpha=0.2);
plt.xlabel('z (m)')
plt.ylabel(r'$\sigma_{x,y}$, max[$x$, $y$] (m)')
plt.legend([r'$\sigma_{x,y}$', r'max[$x$, $y$]'])

In [ ]:
lat3 = Lattice('aperture_test')
lat3.add(Aperture('a1', a=0.01, b=0.03) , ds=0.1)
lat3.add(Screen('scr'), ds=0.25, ref_element='beg')

In [ ]:
_=lat3.write_gpt_lines(output_file='aperture3.in')

In [ ]:
D['r_dist:max_r'] = 6
D['n_particle']=2000
P = D.run()
P.pz = 1e3

In [ ]:
G3 = GPT('aperture3.in', initial_particles=P)
G3.set_variables({'ZSTOP':0.25, 'tmax':50e-8, 'Ntout': 200})
G3.run()

In [ ]:
ax = plt.gca()
for ii in range(1, D['n_particle']+1):
    tii = G3.trajectory(ii, data_type='tout')
    ax.plot(tii['z'], tii['x'])
ax = lat.plot_floor()

In [ ]:
print(G3)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for ii in range(1, D['n_particle']+1, 1):
    tii = G3.trajectory(ii, data_type='tout')
    if(tii['z'][-1]>0.1):
        ax.plot(tii['x']*1000, tii['y']*1000, tii['z'], zdir='x', alpha=0.5)
    else:
        ax.plot(tii['x']*1000, tii['y']*1000, tii['z'], zdir='x', alpha=0.1)
        
    
ax.set_xlabel('z (m)');
ax.set_ylabel('x (mm)');
ax.set_zlabel('y (mm)');
ax.set_ylim([-60, 60])
ax.set_zlim([-60, 60])

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.gca()
for ii in range(1, D['n_particle']+1, 1):
    tii = G3.trajectory(ii, data_type='tout')
    if(tii['z'][-1]>0.1):
        ax.plot(tii['x']*100, tii['y']*100, '.')
ax.set_aspect('equal')
ax.set_xlabel('x (cm)')
ax.set_ylabel('y (cm)')

In [ ]:
plt.plot(G3.tout_stat('mean_z'), +G3.tout_stat('sigma_x'))
plt.plot(G3.tout_stat('mean_z'), +G3.tout_stat('sigma_y'))

plt.plot(G3.tout_stat('mean_z'), -G3.tout_stat('sigma_x'), color='tab:blue');
plt.plot(G3.tout_stat('mean_z'), -G3.tout_stat('sigma_y'), color='tab:orange')
plt.xlabel('z (m)')
plt.ylabel(r'$\sigma_{x,y}$, max[$x$, $y$] (m)')
plt.legend([r'$\sigma_{x,y}$', r'max[$x$, $y$]'])

In [ ]:
os.remove('aperture2.in')
os.remove('aperture3.in')